In [3]:
import os, json 
from sentence_transformers import SentenceTransformer, util
import numpy as np 

c:\Users\91790\Desktop\Vedaz\vedenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data 

In [4]:
try: 
    with open("data/mock_user_data.json", "r") as f:
        user_data = json.load(f)
except FileNotFoundError:
    print("File not found. Please ensure 'mock_user_data.json' exists in the 'data' directory.")
    user_data = []

In [6]:
user_data[0]

{'user_id': 'user_001',
 'chat_history': [{'sender': 'user',
   'message': "Hi, I'm not sure where to start."},
  {'sender': 'bot',
   'message': "Hello! I'm here to help. Please tell me what's on your mind."},
  {'sender': 'user',
   'message': "Well, everything at work just feels like a dead end. My manager is difficult, and I feel so unmotivated. I'm worried about my salary too. I just need to know if I should start looking for something else."}]}

In [7]:
try:
    with open("data/mock_astrologer_data.json", "r") as f:
        astrologer_data = json.load(f)
except FileNotFoundError:
    print("File not found. Please ensure 'mock_astrologer_data.json' exists in the 'data' directory.")
    astrologer_data = []

In [8]:
astrologer_data[0]

{'id': 1,
 'name': 'Astro Vani',
 'specialties': 'love, relationships, marriage, soulmates, compatibility',
 'description': 'With over 15 years of experience, Astro Vani specializes in romantic compatibility, helping you understand your relationships and find your true life partner. She offers detailed readings on love and marriage.'}

### Prepare Data for Semantic Matching

We will transform our raw data into a clean format. 
1. Combine the details of each astrologer into a single descriptive text
2. Select a user and combine their chat messages into a single, coherent query 

In [9]:
astrologer_profiles = [
    f"Astrologer Name: {a['name']}. Specialties: {a['specialties']}. Description: {a['description']}" for a in astrologer_data
]

In [10]:
astrologer_profiles[0]

'Astrologer Name: Astro Vani. Specialties: love, relationships, marriage, soulmates, compatibility. Description: With over 15 years of experience, Astro Vani specializes in romantic compatibility, helping you understand your relationships and find your true life partner. She offers detailed readings on love and marriage.'

In [11]:
# Select a user to recommend astrologers for
user_to_recommend_for = user_data[0]
user_id = user_to_recommend_for['user_id']

# Combine 
chat_messages = [
    message['message'] for message in user_to_recommend_for['chat_history'] if message['sender'] == 'user'
]
user_query = " ".join(chat_messages)

In [12]:
user_query

"Hi, I'm not sure where to start. Well, everything at work just feels like a dead end. My manager is difficult, and I feel so unmotivated. I'm worried about my salary too. I just need to know if I should start looking for something else."

### Embeddings Generation 

We will now use a pre-trained Sentence Transformer model to convert our text data into numerical vectors, also called Embeddings. These capture the semantic meaning of the text, allowing us to compare them mathematically. 

In [13]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [15]:
# Generate Embeddings for astrologer profiles
astrologer_embeddings = model.encode(astrologer_profiles, convert_to_tensor=True)

In [16]:
# Generate embeddings for user query
user_query_embedding = model.encode(user_query, convert_to_tensor=True)

In [20]:
print("User Query Embedding Shape:", user_query_embedding.shape)
print("Astrologer Embeddings Shape:", astrologer_embeddings.shape)

User Query Embedding Shape: torch.Size([384])
Astrologer Embeddings Shape: torch.Size([10, 384])


### Calculating the Best Matches

We will now retrieve the most relevant matches based on the user query. We will use cosine similarity for this purpose. 

In [21]:
# Calculate cosine similarities
cosine_scores = util.cos_sim(
    user_query_embedding, astrologer_embeddings
)

In [22]:
cosine_scores

tensor([[-0.0123,  0.0950,  0.0692,  0.0065,  0.0415,  0.0624,  0.1066,  0.0151,
          0.0681,  0.0177]])

In [23]:
scores = cosine_scores[0]

In [25]:
# Combine the scores with the astrologer profiles
results = []
for i, score in enumerate(scores):
    results.append(
        {
            "score": score.item(),
            "astrologer": astrologer_data[i]
        }
    )

# Sort the results based on the score
results.sort(key=lambda x: x['score'], reverse=True)

In [28]:
print("--- Top 5 Ranked Astrologers ---")
for r in results[:5]:
    print(f"Astrologer: {r['astrologer']['name']}  ||  Score: {r['score']:.4f}")

--- Top 5 Ranked Astrologers ---
Astrologer: Financial Astrologer Ravi  ||  Score: 0.1066
Astrologer: Career Guru Ramesh  ||  Score: 0.0950
Astrologer: Jyotish Anjali  ||  Score: 0.0692
Astrologer: Acharya Krish  ||  Score: 0.0681
Astrologer: Rishi Manish  ||  Score: 0.0624


**We can also Re-rank these results to further improve the quality of recommendation, but it would involve usage of another Re-ranker model which might increase the run-time as well as resource usage for the chatbot.**

### Let's see the Results

In [29]:
top_n = 2 # Number of top astrologers to recommend

In [32]:
def recommend_astrologer(
    user_profile, model, astrologer_data, astrologer_embedding, top_n
):
    """
    Generate recommendations for a user based on their query and astrologer profiles.

    Args:
        user_profile (dict): User profile containing user_id and chat history.
        model (SentenceTransformer): Pre-trained model for generating embeddings.
        astrologer_data (list): List of astrologer profiles.
        astrologer_embedding (Tensor): Pre-computed embeddings for astrologer profiles.
        top_n (int): Number of top recommendations to return.
    
    Returns:
        None: Prints the recommendation report.
    """
    # Process user 
    user_id = user_profile['user_id']
    chat_messages = [
        message['message'] for message in user_profile['chat_history'] if message['sender'] == 'user'
    ]
    user_query = " ".join(chat_messages)

    if not user_query:
        print(f"No user query found for user {user_id}.")
        return 
    
    # Generate embeddings for user query
    u_embedding = model.encode(
        user_query, convert_to_tensor=True
    )

    # Calculate cosine similarities
    cosine_scores = util.cos_sim(
        u_embedding, astrologer_embedding
    )[0]

    results = []
    for i, score in enumerate(cosine_scores):
        results.append(
            {
                "score": score.item(),
                "astrologer": astrologer_data[i]
            }
        )
    # Sort the results based on the score
    results.sort(key=lambda x: x['score'], reverse=True)

    # Display Recommendations
    print("=" * 70)
    print(f"👤 Recommendation Report for User ID: {user_id}")
    print("-" * 40)
    print("Detected User Query:")
    print(f'   "{user_query}"')
    
    print("\n✨ Top 3 Astrologer Recommendations ✨")
    
    top_n = 3
    for i, result in enumerate(results[:top_n], start=1):
        astrologer = result['astrologer']
        score = result['score']
        print(f"   {i}. {astrologer['name']} (Score: {score:.2f})")
        print(f"      💡 Matched Specialties: {astrologer['specialties']}")
    print("=" * 70 + "\n")

In [36]:
print("--- Generating recommendation for a user concerned about their Love life... ---")
recommend_astrologer(
    user_data[1], 
    model, 
    astrologer_data,
    astrologer_embeddings,
    top_n
)

--- Generating recommendation for a user concerned about their Love life... ---
👤 Recommendation Report for User ID: user_002
----------------------------------------
Detected User Query:
   "Hey... so all my friends are getting engaged or married and I'm just... here. I've tried everything but nothing seems to work out. I'm starting to feel really alone and wondering if I'll ever find someone special."

✨ Top 3 Astrologer Recommendations ✨
   1. Astro Vani (Score: 0.27)
      💡 Matched Specialties: love, relationships, marriage, soulmates, compatibility
   2. Rishi Manish (Score: 0.27)
      💡 Matched Specialties: marriage, matchmaking, relationship conflicts
   3. Star Seer Priya (Score: 0.19)
      💡 Matched Specialties: love, career, personal growth, life path



In [37]:
print("--- Generating recommendation for a user concerned about life decisions... ---")
recommend_astrologer(
    user_data[2], 
    model, 
    astrologer_data,
    astrologer_embeddings,
    top_n
)

--- Generating recommendation for a user concerned about life decisions... ---
👤 Recommendation Report for User ID: user_003
----------------------------------------
Detected User Query:
   "Hello. My business has hit a wall. Sales have been down for months and I'm pouring all my savings into it. My family is getting worried. I need to know if I should keep pushing or if it's a lost cause."

✨ Top 3 Astrologer Recommendations ✨
   1. Financial Astrologer Ravi (Score: 0.06)
      💡 Matched Specialties: stock market, trading, wealth management, finance
   2. Healer Divya (Score: 0.02)
      💡 Matched Specialties: emotional healing, stress, anxiety, wellness
   3. Jyotish Anjali (Score: -0.00)
      💡 Matched Specialties: health, wellness, mental peace, spiritual growth



In [40]:
print("--- Generating recommendation for a user concerned about mental piece... ---")
recommend_astrologer(
    user_data[5], 
    model, 
    astrologer_data,
    astrologer_embeddings,
    top_n
)

--- Generating recommendation for a user concerned about mental piece... ---
👤 Recommendation Report for User ID: user_006
----------------------------------------
Detected User Query:
   "Hi. I'm not sure I believe in all this, but my friend insisted. I just want to know if there's anything I should be aware of for the next year? Like, any big changes in my job or personal life?"

✨ Top 3 Astrologer Recommendations ✨
   1. Jyotish Anjali (Score: 0.17)
      💡 Matched Specialties: health, wellness, mental peace, spiritual growth
   2. Healer Divya (Score: 0.16)
      💡 Matched Specialties: emotional healing, stress, anxiety, wellness
   3. Career Guru Ramesh (Score: 0.14)
      💡 Matched Specialties: career, finance, business, success, investments

